In [6]:
import pandas as pd
import pickle
import numpy as np
import re

In [7]:
#setting up the mainframe
df1 = pd.read_csv("watson_art1.csv", delimiter = "|", header = None)
df2 = pd.read_csv("watson_art2.csv", delimiter = "|", header = None)
df3 = pd.read_csv("watson_art3.csv", delimiter = "|", header = None)
df4 = pd.read_csv("watson_art4.csv", delimiter = "|", header = None)

frames = [df1, df2, df3, df4]
df_main = pd.concat(frames, ignore_index= True)

In [8]:
# drop duplicates
df_main = df_main.drop_duplicates(keep = "last", ignore_index = True)

In [9]:
#setting up cateogries
category = df_main[5]
caty = category.str.split("/",expand=True,)
df_main[6] = caty[3]
df_main[7] = caty[4]

In [10]:
#setting dates straight
time_indy = df_main[2]
time_indy = time_indy.str.split(",",expand=True)

df_main[2] = time_indy[0]
df_main[8] = time_indy[1]

df_main[2] = pd.to_datetime(df_main[2], format="%d.%m.%y")
df_main[9] = df_main[2].dt.year

df_main = df_main[df_main[9] != 2021]
df_main = df_main.drop([9], axis = 1)
df_main = df_main.reset_index(drop = True)

In [12]:
#remove columns without title, text or url
df_main = df_main.dropna(subset=[0])
df_main = df_main.dropna(subset=[5])
df_main = df_main.dropna(subset=[4])

In [13]:
#reorder columns
df_main = df_main[[0,1,2,8,3,4,5,6,7]]

In [14]:
df_main[9] = df_main[6]
df_main[10] = df_main[7]

In [15]:
#replace topics with ressort-categories
with open("ressorts_cat.bin", "rb") as data:
    ressorts = pickle.load(data)

df_main[9].replace(ressorts, inplace=True)
df_main[10].replace(ressorts, inplace=True)

In [16]:
#cancel out doubles
df_main[11] = np.where(df_main[9] == df_main[10], "", df_main[10])
df_main = df_main.drop(columns = 10)

In [17]:
#delete unworkables
with open("to_delete.bin", "rb")as data:
    to_delete = pickle.load(data)
    
df_main = df_main[~df_main[8].isin(to_delete)]
df_main = df_main[~df_main[7].isin(to_delete)]

In [18]:
#watson often shows at the end of an article another article. As those texts are comprised in the same
#css class, one cannot distinguish those in the scraper. Instead, it needs a manual inspection. Often, the same
#articles are promotet in the same time span and as the scraping for watson was done within a couple of days,
#so the diversity of the different promoted articles is relatively stron


end_to_delete = ["Hunderte Fasnächtlerinnen und Fasnächtler versammelten sich am Montag in Ensiedeln. Dazu gesellten sich zahlreiche Zuschauerinnen und Zuschauer. Zunächst schritt die Polizei nicht ein, verteilte dann aber vereinzelte Bussen. Hunderte Fasnächtlerinnen und Fasnächtler haben sich am Montag zum «Sühudiumzug» in Einsiedeln getroffen. Dies trotz strikter Corona-Massnahmen, die in der ganzen Schweiz gelten. «CH Media» berichtet von Tausenden Zuschauern, die dem Umzug im Schwyzer Pilgerort beiwohnten. Es habe kaum weniger Personen als in vergangenen Jahren gehabt. >> Coronavirus: Alle News im Liveticker  Die Schwyzer Polizei liess die Fasnächtlerinnen und Fasnächtler gewähren. Der Sprecher der Kapo Schwyz, David …",
                "Alleine die beiden Firmen Pfizer und Moderna gehen von Umsätzen für die Coronavakzine von knapp 27 Milliarden Dollar aus. Derweil werden in ärmeren Ländern bis im Sommer nur gerade 3 Prozent der Bevölkerung geimpft. Viele wundern sich noch immer. In weniger als einem Jahr wurden mehrere Impfstoffe gegen Corona entwickelt. Der bisherige Rekord in der Herstellung eines Impfstoffs beträgt vier Jahre. Dabei geht vergessen, wie viel staatliche Gelder in die Entwicklung gesteckt wurden. Schätzungen gehen von 86.5 Milliarden Euro aus, wie eine Studie der Kenup Foundation zeigt, einer europäischen Nichtregierungsorganisation. Allein die USA haben über 13 Milliarden Dollar investiert, die EU und Grossbritannien …",
                "Das Eishockey weltweit kämpft mit dem Coronavirus. Wie im Schweizer Eishockey kommt es auch in der NHL zu diversen Spielverschiebungen. Der Verdacht erhärtet sich, dass es auch auf dem Eis zu Übertragungen der Krankheit kommen kann. Nicht nur im Schweizer Eishockey, auch in der NHL stellt sich das Coronavirus als grosses Problem heraus. Rund 100 Spieler wurden bereits auf die Covid-19-Liste gesetzt, jüngst auch Nico Hischier. Das bedeutet nicht unbedingt, dass sie sich mit dem Virus infiziert haben, es kann auch sein, dass sie Kontakt hatten mit einer positiv getesteten Person oder aus anderen Gründen in Quarantäne mussten. Aktuell ist bei vier Teams der Spielbetrieb eingestellt: den New Jersey Devils, den Buffalo Sabres, …",
                "Mit 117 Jahren führt sie die Liste der ältesten Frauen Europas an – und hat gerade erst eine Infektion mit dem Coronavirus überstanden. Schwester André feierte am Donnerstag ihren Geburtstag in einem Altersheim in Toulon in Südfrankreich. Sie habe am Morgen mit ihrer Familie per Videokonferenz sprechen können, sagte David Tavella, Kommunikationsdirektor des Heims, der Deutschen Presse-Agentur in Paris. Später sei der Bischof von Toulon gekommen. Schwester André habe dann an einer Messe in …",
                "Ein beträchtlicher Teil der Schweizer Bevölkerung will bei der Corona-Impfung zuwarten oder lehnt sie ganz ab. Die Skeptiker vergessen, wie segensreich Impfungen für die Menschheit waren. Und was auf dem Spiel steht. Bundeshaus in Bern während der Wintersession im Dezember. Zufällige Begegnung in der Wandelhalle mit einem Bundesrats-Mitarbeiter. Er erzählte mir eine Episode, die mir zu denken gab. Bei einer Visite mit seiner kleinen Tochter bei der Kinderärztin habe diese ihm und seiner Frau dafür gedankt, dass sie ihren Nachwuchs bedenken- und diskussionslos impfen liessen. >> Coronavirus: Alle News im Liveticker  Das sei heute nicht mehr selbstverständlich, habe die Ärztin gesagt. Etwa mit der …",
                "Reiche Industrienationen dürften ihre Bevölkerung bereits dieses Jahr durchgeimpft haben. Dabei macht der Alleingang westlicher Nationen weder ethisch, noch gesundheitlich, noch wirtschaftlich Sinn. Wenige haben es geglaubt, nun ist es Realität: Nach nicht einmal 12 Monaten Pandemie gibt es bereits eine Handvoll an Impfstoffen, die gegen Covid-19 eingesetzt werden können.  Bei aller Freude über dieses medizinische Wunder scheint jedoch vergessen zu gehen, dass Impfstoffe alleine keine Pandemien stoppen. Sie in die Arme der Menschen zu spritzen hingegen schon. Und zwar in die Arme möglichst aller Menschen in allen Ländern, sonst gibt es keine Herdenimmunität und ohne Herdenimmunität kein …",
                "Europäische Staaten erleichtern das Reisen für Geimpfte. Auch der Kanton Graubünden bevorteilt immune Personen, eine Schweizer Strategie gibt's aber noch nicht. Wetten, dass Sie im Sommer nicht nach Italien in die Ferien reisen, ohne gegen Covid-19 geimpft zu sein? Und ohne Impfpass auch nicht nach Neuseeland fliegen, das argwöhnisch darüber wacht, keine Infizierten ins Land zu lassen? Eine Reise- und Quarantänefreiheit wird im Sommer ohne Impfung wohl kaum zu haben sein. In diese Richtung zeigen die Meldungen, welche dieser Tage beispielsweise in den britischen Medien zu lesen sind. Demnach denkt die britische Regierung über Impfpässe nach, die es …",
                "Covid-19, die vom neuartigen Coronavirus SARS-CoV-2 verursachte Erkrankung, und die saisonale Grippe, fachsprachlich Influenza genannt, haben einiges gemeinsam: Beide sind Atemwegserkrankungen, die von einem Virus verursacht werden, beide werden hauptsächlich über Tröpfcheninfektion oder direkten Kontakt verbreitet, beide weisen sehr unterschiedliche Schweregrade auf, die von symptomlosen oder milden Formen bis zu lebensbedrohlichen oder gar tödlichen Verläufen reichen.  Allerdings gibt es auch …",
                "Die Schweiz stimmt über ein Verhüllungsverbot ab in einer Zeit, in der wir uns alle verhüllen. Die Initiative erlaubt das explizit, weshalb Kritiker sie als heuchlerisch bezeichnen. Als Karin Keller-Sutter (FDP) am 19. Januar vor den Medien das Nein des Bundesrats zur Volksinitiative des Egerkinger Komitees begründete, trug sie einen Mundschutz. Farbe und Muster waren auf ihren Blazer abgestimmt. Es war wohl primär ein modisches Statement der St. Gallerin, aber eigentlich unterstrich sie damit einen wesentlichen Grund für ein Nein. Die Abstimmung über das Verhüllungsverbot findet in einer Zeit statt, in der wir uns alle verhüllen. In öffentlich zugänglichen Innenräumen …",
                "Die EU hat den Impfstart verschlafen: Zu wenig Impfstoff, zu spät bestellt, zu wenig Geld auf den Tisch geblättert – die Klagen gegen die EU sind mannigfaltig. Die EU-Kommissionspräsidentin Ursula von der Leyen musste am Sonntag im «ZDF Heute Journal» dem Moderator Claus Kleber Rede und Antwort stehen. Wie könne es sein, dass die USA und Grossbritannien die EU abgehängt haben? Notabene zwei Regierungen, die von (ehemaligen) Corona-Skeptikern angeführt wurden? Von der Leyen entgegnete: «Dann …",
                "20 Jahre lang hat niemand an Katalin Karikós Forschungen geglaubt, weitergemacht hat die Biochemikerin trotzdem. Dank ihrer Beharrlichkeit haben wir nun einen Impfstoff gegen Covid-19. Den Nobelpreis dafür will sie aber trotzdem nicht unbedingt.  Katalin Karikó hat sich noch nie darauf versteift, was andere denken oder tun. Hätte sie es getan, würde sie heute nicht als die Mutter der mRNA-Technik gefeiert.  Es ist jener unerschütterliche Glaube an die Sache und an die eigene Fähigkeit, eines Tages ihre Nützlichkeit beweisen zu können. Diese stille Beharrlichkeit, die nichts braucht ausser Zeit und immer wieder neuen Anläufen, die Karikó zu einer so wahrhaften Wissenschaftlerin machen.  Ein bisschen erinnert sie an Marie Curie, wie sie …",
                "Dieser Text hat etwas Paradoxes: Er informiert dich, liebe Leserin, lieber Leser, über eine der zahlreichen Gefahren der Corona-Pandemie, nämlich jene, dass zu viel Information über die Corona-Pandemie schädlich sein kann. Du bist also gewarnt – wenn du weiterlesen willst, geschieht dies auf dein eigenes Risiko.  In Panik verfallen und diesen Beitrag aus Sorge über dein Wohlergehen gleich wieder wegklicken musst du gleichwohl nicht. Nur wenn du dir jeden Tag mindestens drei Stunden lang …",
                "Walter Krämer erzählt, wie mit fragwürdigen Studien Panik gemacht wird, während wir uns eigentlich vor ganz anderen Dingen fürchten sollten. Herr Krämer, Sie sind Professor für Wirtschafts- und Sozialstatistik und entlarven als solcher gern zwielichtige Datenanalysen. Haben Sie derzeit eine Lieblingsstatistik? Walter Krämer: Ja. Das ist die Tabelle der Fussball-Bundesliga. Ich wohne direkt neben dem BVB-Stadion und bin seit mehr als einem Dutzend Jahren BVB-Aktionär. Sie haben die Deutschen mal als «Volk von Innummeraten» (Zahlen-Analphabeten) bezeichnet und ihre bedauerliche Schwäche Goethe, dem dichtenden …",
                "Restaurantbesuche, Kinoabende, Reisen: Sollen Geimpfte dürfen, was anderen in der Corona-Pandemie noch versagt ist? In der Schweiz steht man dem eher kritisch gegenüber, doch in einigen europäischen Ländern ist das bereits Realität. Ein kleiner Überblick. Dänemark legte vergangene Woche Pläne für einen digitalen Ausweis mit Impfdaten vor, um Dienstreisen in Corona-Zeiten zu erleichtern - und letztlich vielleicht auch die sorgenfreie Teilnahme an Konzerten oder Sportveranstaltungen zu …",
                "Momentan werden fast nur Erwachsene gegen das Coronavirus geimpft. Nun laufen erste Tests für die Corona-Impfung für Jugendliche. Das Wichtigste in 5 Punkten. Zähe zieht sich der Winter dahin. Obschon die Corona-Fallzahlen weiter sinken, wird es wohl noch eine Weile dauern, bis wieder Normalität einkehrt. Zwar ist die Impfkampagne angelaufen, bis jedoch jeder geimpft werden kann, der es möchte, wird es noch mindestens bis im Frühsommer dauern.  Um das Coronavirus endgültig zu besiegen, braucht es eine «Herdenimmunität». Ging man zunächst davon aus, dass dafür rund 60 bis 70 Prozent der Bevölkerung immun sein muss, rechnet man mittlerweile mit 70 …",
                "Donald Trumps Amtszeit war geprägt von Lügen, Korruption und der Anstiftung zur Gewalt. Vier turbulente Trump-Jahre im Spiegel der Karikaturisten. Hinweis für die geschätzten User und Userinnen: Wenn die Tweets in diesem Artikel nicht sofort angezeigt werden, zähle wahlweise laut oder leise auf fünf und scrolle erst dann weiter. Donald Trump verliert mit Joe Bidens Amtseinführung seine Immunität, er kann somit vor ein ordentliches Gericht gestellt werden. Beobachter glauben aber, dass er eher nicht ins Gefängnis muss. Ungemütlich wird es für ihn trotzdem. Viele Fälle sind noch offen: Amtsmissbrauch, Behinderung der Justiz und diverse …",
                "In der Schweiz sind Lockerungen derzeit noch nicht in Sicht. Das hat Bundesrat Alain Berset an der gestrigen Medienkonferenz noch einmal betont. Anders sieht es im nahen Ausland aus. Dort haben einige Länder ihre Massnahmen teilweise entschärft. Und das, obwohl etwa in Italien die Wochen-Inzidenz höher ist als hierzulande. Anders sieht es in Polen aus: Dort lockert man trotz tieferer Inzidenz eher zurückhaltend. In Österreich kommt es gar zu Lockerungen und Verschärfungen zugleich.  So sieht die …",
                "Auf einem verschneiten Golfplatz in Helsinki ist ein atemberaubendes Schneespektakel in den Boden gestampft worden.  Man glaub es kaum: Dieses Mandala im Schnee ist nicht maschinell eingestampft worden. Eine Reihe von Freiwilligen ist für das geometrische Schneekunstwerk verantwortlich. Die Gruppe folgte einer ausgedruckten Karte und benutzte Seile, um auf dem schneebedeckten Golfplatz in Helsinki perfekte Kreise nachzuzeichnen.  Der Zeitaufwand für die riesige Schneeflocke mit einem Durchmesser von 160 Metern war entsprechend gross. Ganze drei Stunden bei -10 Grad sei die Gruppe dafür auf dem verschneiten …",
                "Alle Erkenntnisse über den Ursprung des Coronavirus Sars-CoV-2 deuten nach Angaben von Experten der Weltgesundheitsorganisation (WHO) auf ähnliche Viren in Fledermäusen. Nach Abschluss der gemeinsam mit chinesischen Experten vorgenommenen Untersuchungen in der Metropole Wuhan in Zentralchina sagte der Chef des Teams, Peter Ben Embarek, am Dienstag vor Journalisten, der «wahrscheinlichste Weg» der Übertragung auf den Menschen sei von Fledermäusen ausgehend über ein anderes Tier als …",
                "Das Pflegepersonal ist am Anschlag und drei Chefärzte kritisieren die Politik. Wie bedroht ist das Schweizer Gesundheitswesen durch das Coronavirus? Drei Chefärzte in leitender Position in den drei grössten Spitälern des Kantons Zürich wehren sich gegen die aktuelle Corona-Politik des Bundes und der Kantone. Die grundsätzliche Einschätzung zusammengefasst: Bei den drei Ärzten handelt es sich um Urs Karrer, Chefarzt Infektiologie am Kantonsspital Winterthur, Professor Huldrych Günthard, stellvertretender Direktor an der Klinik für Infektionskrankheiten am Unispital Zürich und Gerhard Eich, Chef-Infektiologe am Triemli. Sie meldeten sich diese …",
                "In Bern macht eine Gruppe von Corona-Skeptikern Schulhäuser unsicher, dringt ohne Maske in Lehrerzimmer ein und verteilt Flyer mit Falschinformationen. Hinter der Aktion steckt ein bekannter Berner Verschwörungs-Anhänger. Jetzt greift die Polizei durch.  Corona-Leugner machen inzwischen selbst vor Kindern nicht Halt. In Bern sind seit letzter Woche Covid-Skeptiker in mehrere Schulhäuser eingedrungen und haben Corona-Falschinformationen verbreitet. «Sie haben Flyer an die Schüler verteilt und sind ohne Schutzmaske in das Lehrerzimmer eingedrungen. Das ist total daneben», sagt Giuliano Picciati, Leiter der Schule Munzinger im Berner Mattenhof-Quartier. Nachdem er von Lehrern alarmiert worden sei, habe er die Eindringlinge vom Schulhausplatz …",
                "Die Online-Petition gegen den Lockdown hat bereits mehr als 170'000 Unterzeichnende. Recherchen von watson zeigen, dass diese Zahl mit Vorsicht zu geniessen ist. Das Generalsekretariat der SVP Schweiz beschwichtigt. Die Online-Petition lockdown-stop.ch sorgt wegen der relativ hohen Zahl von Unterzeichnenden für Schlagzeilen. Dass innert drei Wochen über 100'000 Personen Unterstützung signalisierten, stiess aber auch auf Misstrauen.  Recherchen von watson zeigen nun: Die von rechtsbürgerlicher Seite lancierte Online-Petition  konnte einfach manipuliert werden, um die Zahl der Unterzeichnenden in die Höhe zu treiben. Das Generalsekretariat der SVP Schweiz versuchte am Mittwoch, den Manipulationsverdacht zu …",
                "War das fasnächtliche Treiben in Einsiedeln gestern einigermassen unter Kontrolle, musste in der Nacht ein grösseres Polizeiaufgebot ausrücken. 5 Punkte zum Fasnacht-Debakel. Schon gestern morgen zeichnete sich grosse Fasnachts-Lust ab als Hunderte von Menschen nach Einsiedeln pilgerten, um dort am «Suhüdiumzug» teilzunehmen. Die Schwyzer Polizei lässt sie zuerst gewähren – da es sich um keine Veranstaltung handle – greift dann aber nach Verstössen gegen die Masken- und Abstandspflicht ein. Um 13:22 Uhr ruft die Schwyzer Polizei die Menschen auf, Ansammlungn zu meiden und auf fasnächtliches Treiben zu verzichten: Vergebens. Die Neuigkeiten über den «Suhüdiumzug» …",
                "Bereits im Juni schlug die Taskforce vor, flächendeckende Sequenzierungen vorzunehmen. Das BAG setzte den Vorschlag jedoch nicht um. Weshalb, will das BAG nicht sagen. Vor 22 Tagen beschloss der Bundesrat strikte Massnahmen im Kampf gegen das Coronavirus. Die neuen Regeln sind einschneidend: Schweizerinnen und Schweizer dürfen sich höchstens noch in Gruppen von fünf Personen treffen, Einkaufsläden bleiben bis mindestens Ende Februar zu. Vielleicht sogar noch länger. Bundesrat Alain Berset sagte diesen Mittwoch in Bern: «Man kann nicht davon ausgehen, dass Ende Februar grossflächige Lockerungen möglich sind.» Der Grund für die Massnahmen: Das Coronavirus ist …"
                ]

In [19]:
for element in end_to_delete:
    df_main[4] = df_main[4].str.replace(element, "")

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [20]:
restliches_or = df_main[4].str.rsplit("(", expand=True, n=1)
restliches = restliches_or[1].str.rsplit(")", expand=True, n=1)

In [21]:
restliches[2] = np.where(restliches[1] == " ", restliches[0], df_main[3])

In [22]:
restliches[3] = np.where(restliches[2].str.len() < 20, restliches[2], df_main[3])

In [23]:
restliches_or[2] = restliches[0]
restliches_or[3] = restliches[1]
restliches_or[4] = df_main[4]
df_main[4] = np.where(restliches_or[3] == " ", restliches_or[0], df_main[4])

In [24]:
df_main[3] = restliches[3]
df_main[4] = df_main[4].str.strip()


In [25]:
df_main[4] = df_main[4].str.replace("http\S+", "",regex = True)
df_main[4] = df_main[4].str.replace("Bild", "")

In [26]:
df_main[12] = "watson"

In [28]:
df_main.to_csv("watson_artikel.csv", header = None, sep = "|", index = False)

In [27]:
df_main

,0,1,2,8,3,4,5,6,7,9,11,12
0,PIMS: Die Corona-Folge bei Kindern und was du ...,Das PIM-Syndrom ist eine Spätfolge von Covid-1...,2020-12-31,17:33,Agentur,Hohes Fieber während mehrerer Tage ist ein kla...,https://www.watson.ch/schweiz/wissen/757130367...,schweiz,wissen,schweiz,wissenschaft,watson
1,So hat sich die (falsche) Meldung über den «Im...,NaN,2020-12-31,14:10,Agentur,"Diese Woche meldeten diverse Medien, dass ein ...",https://www.watson.ch/schweiz/luzern/178864959...,schweiz,luzern,schweiz,lokales,watson
2,Wir haben es schon fast vergessen: So locker w...,"Die Fallzahlen waren im Sommer so tief, dass w...",2020-12-31,13:39,cki,"Die Fallzahlen waren im Sommer so tief, dass w...",https://www.watson.ch/schweiz/coronavirus/4182...,schweiz,coronavirus,schweiz,nachrichten,watson
3,Hier ist es: Das neue Bundesratsfoto! Wir habe...,Das Bundesratsfoto 2021 ist da. Es zeigt die S...,2020-12-31,12:00,Agentur,Das Bundesratsfoto 2021 ist da. Es zeigt die S...,https://www.watson.ch/schweiz/analyse/57111059...,schweiz,analyse,schweiz,vermischtes,watson
4,Heute vor einem Jahr wurde gemeldet: «Mysteriö...,«Mysteriöse Lungenkrankheit in Wuhan ausgebroc...,2020-12-31,10:38,Agentur,Hier soll alles seinen Lauf genommen haben: de...,https://www.watson.ch/schweiz/medien/984670088...,schweiz,medien,schweiz,kultur,watson
...,...,...,...,...,...,...,...,...,...,...,...,...
3098,Ausbreitung grösser als angenommen? 17 neue Fä...,NaN,2020-01-19,07:28,sda/dpa,Mehr Fälle als bisher angenommen: Chinesen sch...,https://www.watson.ch/international/wissen/818...,international,wissen,international,wissenschaft,watson
3099,Vier neue Fälle der rätselhaften Lungenkrankhe...,NaN,2020-01-18,12:55,sda/dpa,Eine Reisende mit Atemschutzmaske in Peking. :...,https://www.watson.ch/international/china/3787...,international,china,international,,watson
3100,Mysteriöse Lungenkrankheit: Zweiter Todesfall ...,NaN,2020-01-17,06:47,sda/dpa,In China ist ein zweiter Patient an einer myst...,https://www.watson.ch/international/china/2106...,international,china,international,,watson
3101,Mysteriöse neue Lungenkrankheit fordert erstes...,NaN,2020-01-11,06:54,sda/dpa,Erstmals ist ein Patient an der rätselhaften n...,https://www.watson.ch/international/gesundheit...,international,gesundheit,international,vermischtes,watson
